<a href="https://colab.research.google.com/github/NakkaBhavith/450/blob/main/Alzheimers(logistic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/AI Data/oasis_longitudinal.csv")
data.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [ ]:
data.drop(['Subject ID','MRI ID','Hand','Visit','MR Delay'], axis = 1, inplace = True)
# Checking for null values
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Group   373 non-null    object 
 1   M/F     373 non-null    object 
 2   Age     373 non-null    int64  
 3   EDUC    373 non-null    int64  
 4   SES     354 non-null    float64
 5   MMSE    371 non-null    float64
 6   CDR     373 non-null    float64
 7   eTIV    373 non-null    int64  
 8   nWBV    373 non-null    float64
 9   ASF     373 non-null    float64
dtypes: float64(5), int64(3), object(2)
memory usage: 29.3+ KB
None


In [ ]:
pd.isnull(data).sum() 

Group     0
M/F       0
Age       0
EDUC      0
SES      19
MMSE      2
CDR       0
eTIV      0
nWBV      0
ASF       0
dtype: int64

In [ ]:
data = data.dropna(axis=0, how='any')
pd.isnull(data).sum()

Group    0
M/F      0
Age      0
EDUC     0
SES      0
MMSE     0
CDR      0
eTIV     0
nWBV     0
ASF      0
dtype: int64

In [ ]:
# Converting the categorical variable into numerical
varlist1 =  ['M/F']
varlist2 = ['Group']

# Defining the map function
def binary_map1(x):
    return x.map({'M': 1, "F": 0})
def binary_map2(y):
    return y.map({'Demented' : 1, 'Nondemented' : 0, 'Converted' : 0})

data[varlist1] = data[varlist1].apply(binary_map1)
data[varlist2] = data[varlist2].apply(binary_map2)

data

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,1,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,0,1,88,14,2.0,30.0,0.0,2004,0.681,0.876
5,0,0,88,18,3.0,28.0,0.0,1215,0.710,1.444
6,0,0,90,18,3.0,27.0,0.0,1200,0.718,1.462
7,0,1,80,12,4.0,28.0,0.0,1689,0.712,1.039
...,...,...,...,...,...,...,...,...,...,...
368,1,1,82,16,1.0,28.0,0.5,1693,0.694,1.037
369,1,1,86,16,1.0,26.0,0.5,1688,0.675,1.040
370,0,0,61,13,2.0,30.0,0.0,1319,0.801,1.331
371,0,0,63,13,2.0,30.0,0.0,1327,0.796,1.323


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Applying scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['Age', 'EDUC', 'SES', 'MMSE', 'eTIV', 'ASF']
data[num_vars] = scaler.fit_transform(data[num_vars])

data

,Group,M/F,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,1,0.710526,0.470588,0.25,0.884615,0.0,0.981069,0.696,0.009845
1,0,1,0.736842,0.470588,0.25,1.000000,0.0,1.000000,0.681,0.000000
5,0,0,0.736842,0.705882,0.50,0.923077,0.0,0.121381,0.710,0.798875
6,0,0,0.789474,0.705882,0.50,0.884615,0.0,0.104677,0.718,0.824191
7,0,1,0.526316,0.352941,0.75,0.923077,0.0,0.649220,0.712,0.229255
...,...,...,...,...,...,...,...,...,...,...
368,1,1,0.578947,0.588235,0.00,0.923077,0.5,0.653675,0.694,0.226442
369,1,1,0.684211,0.588235,0.00,0.846154,0.5,0.648107,0.675,0.230661
370,0,0,0.026316,0.411765,0.25,1.000000,0.0,0.237194,0.801,0.639944
371,0,0,0.078947,0.411765,0.25,1.000000,0.0,0.246102,0.796,0.628692


In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(data, train_size = 0.5, test_size = 0.5, random_state = 10)

In [ ]:
y_train = df_train.pop('Group')
X_train = df_train
y_test = df_test.pop('Group')
X_test = df_test

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
y_pred_train = lr_model.predict(X_train)
y_pred_test = lr_model.predict(X_test)

In [ ]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.22549380840084865
Mean square error 0.05084745762711865
Mean absolute error 0.05084745762711865


In [ ]:
#Testing Accuracies
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.21259760138109354
Mean square error 0.04519774011299435
Mean absolute error 0.04519774011299435


In [ ]:
print(" Training Accuracy",lr_model.score(X_train, y_train))
print(" Testing Accuracy",lr_model.score(X_test, y_test))

 Training Accuracy 0.9491525423728814
 Testing Accuracy 0.9548022598870056


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_test)

array([[109,   8],
       [  0,  60]])

In [ ]:
confusion_matrix(y_train, y_pred_train)

array([[104,   6],
       [  3,  64]])

In [ ]:
import pickle
# Save the model
filename = 'model.pkl'
pickle.dump(lr_model, open(filename, 'wb'))

In [ ]:
list_of_columns = data.columns
input_data=pd.DataFrame(columns=list_of_columns)
input_data.drop(['Group'], axis='columns', inplace=True)

input_data.at[0, 'M/F'] = input('enter Gender : ')
input_data.at[0, 'Age'] = int(input('enter Age : '))
input_data.at[0, 'EDUC'] = int(input('enter education : '))
input_data.at[0, 'SES'] = int(input('enter SES : '))
input_data.at[0, 'MMSE'] = int(input('enter MMSE : '))
input_data.at[0, 'CDR'] = float(input('enter CDR : '))
input_data.at[0, 'eTIV'] = int(input('enter eTIV : '))
input_data.at[0, 'nWBV'] = float(input('enter nWBV : '))
input_data.at[0, 'ASF'] = float(input('enter ASF : '))

# Defining the map function
def binary_map(x):
    return x.map({'M': 1, "F": 0})

input_data[varlist1] = input_data[varlist1].apply(binary_map)


input_data['Age']=(input_data['Age']-data['Age'].min())/(data['Age'].max()-data['Age'].min())
input_data['EDUC']=(input_data['EDUC']-data['EDUC'].min())/(data['EDUC'].max()-data['EDUC'].min())
input_data['SES']=(input_data['SES']-data['SES'].min())/(data['SES'].max()-data['SES'].min())
input_data['MMSE']=(input_data['MMSE']-data['MMSE'].min())/(data['MMSE'].max()-data['MMSE'].min())
input_data['CDR']=(input_data['CDR']-data['CDR'].min())/(data['CDR'].max()-data['CDR'].min())
input_data['eTIV']=(input_data['eTIV']-data['eTIV'].min())/(data['eTIV'].max()-data['eTIV'].min())
input_data['nWBV']=(input_data['nWBV']-data['nWBV'].min())/(data['nWBV'].max()-data['nWBV'].min())
input_data['ASF']=(input_data['ASF']-data['ASF'].min())/(data['ASF'].max()-data['ASF'].min())
y_pred =  lr_model.predict(input_data)

if(y_pred[0]==1):
  print('Will suffer with Alzheimers disease')
if(y_pred[0]==0):
  print('Will not suffer with alzheimers disease')

enter Gender : M
enter Age : 87
enter education : 14
enter SES : 2
enter MMSE : 27
enter CDR : 0
enter eTIV : 1987
enter nWBV : 0.696
enter ASF : 0.883
Will not suffer with alzheimers disease
